In [24]:
def save_img(url, name):
        #이미지 url과 저장할 이름을 입력하면 해당 이름으로 저장해주는 함수
        #일단 기본은 10mb까지 저장할 수 있게 해둠
    import requests
    res = requests.get(url)
    res.raise_for_status()
    img = open(name + '.png', 'wb')

    for chunk in res.iter_content(10000000): #byte로 환산해서 얼마만큼 저장해야 하는지 적으면 됨
        img.write(chunk)
    img.close()

In [25]:
import os

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except:
        print ('Error: Creating directory. ' +  directory)

In [26]:
import requests, os, time, random
from selenium import webdriver
import pandas as pd
from tqdm import tqdm_notebook

In [33]:
class StupidSpider:
    def __init__(self, pth='chromedriver.exe'):        
        options = webdriver.ChromeOptions()
        options.add_argument('headless')
        options.add_argument('window-size=1920x1080')
        options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
        self.driver = webdriver.Chrome(pth, options=options)
        self.driver.maximize_window()
    
    def move(self, pth):
        self.driver.get(pth)
        time.sleep(random.randrange(2,4))
    
    def get(self, xpth):
        self.target = self.driver.find_element_by_xpath(xpth)
        return self.target
    
    def gets(self, xpth):
        return self.driver.find_elements_by_xpath(xpth)
    
    def click(self, xpth="$%&"):
        if xpth != "$%&":
            tmp = self.driver.find_element_by_xpath(xpth)
            tmp.click()
            time.sleep(random.randrange(2,4))
        else:
            self.target.click()
            time.sleep(random.randrange(2,4))
    
    def send(self, key, xpth="$%&"):
        if xpth != "$%&":
            tmp = self.driver.find_element_by_xpath(xpth)
            tmp.send_keys(key)
            time.sleep(random.randrange(2,4))
        else:
            self.target.send_keys(key)
            time.sleep(random.randrange(2,4))
    
    def text(self, xpth="$%&"):
        if xpth != "$%&":
            return self.driver.find_element_by_xpath(xpth).text
        else:
            return self.target.text.replace(' ','')
    
    def get_soup(self, html = '$%&'):
        if html != '$%&':
            res = requests.get(html)
            self.soup = bs.BeautifulSoup(res.text, 'html.parser')
            return self.soup
        else:
            html = self.driver.page_source
            self.soup = bs(html, 'html.parser')
            return self.soup

    def soup_select(self, text):
        return self.soup.select(text)
    
    def get_attribute(self, text):
        return self.target.get_attribute(text)
    
    

In [40]:
id_df = pd.read_excel('id_df.xlsx', index_col = 0)
id_df

,회사,차종,모델,id,site
0,현대,i30,i30(PD)(16~20),30555229,encar
1,현대,i30,i30(PD)(16~20),30554651,encar
2,현대,i30,i30(PD)(16~20),29727605,encar
3,현대,i30,i30(PD)(16~20),30316479,encar
4,현대,i30,i30(PD)(16~20),30311314,encar
...,...,...,...,...,...
32793,기아,프라이드,올뉴프라이드(11~14),60539105,kcar
32794,기아,프라이드,올뉴프라이드(11~14),60526564,kcar
32795,기아,프라이드,올뉴프라이드(11~14),60532834,kcar
32796,기아,프라이드,올뉴프라이드(11~14),60534399,kcar


In [41]:
import re

spider = StupidSpider()

for i in range(len(id_df)):
    if id_df.loc[i]['site'] == 'kcar':
        continue
    id_ = id_df.loc[i]['id']
    url = f"http://www.encar.com/dc/dc_cardetailview.do?pageid=dc_carsearch&listAdvType=normal&carid={id_}"
    spider.move('http://www.encar.com/index.do')
    spider.move(url)
    time.sleep(2)
    try: 
        spider.get('//*[@id="carPic"]/img')
    except:
        try : 
            spider.get('//*[@id="carPic"]/span/img')
        except:
            continue
    url_ = spider.get_attribute('src')
    id_regex = re.compile('[0-9]+_001.jpg')
    id_ = id_regex.search(url_).group()
    id_df.at[i, 'id'] = id_[:-8]

<ipython-input-41-eea42e4bc589>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(id_df.iloc[:10]))):


  0%|          | 0/10 [00:00<?, ?it/s]

In [16]:
def save_used_car_img(loc):
    loc['id'] = str(loc['id'])
    if loc['site'] == 'encar':
        for i in range(1,5):
            if loc['id'][3] == '0':
                tmp = '10'
            else:
                tmp = '0' + loc['id'][3]
            url = f"https://ci.encar.com/carpicture/carpicture{tmp}/pic{loc['id'][:4]}/{loc['id']}_00{i}.jpg"
            name = f"{loc['회사']}/{loc['모델']}/{loc['id']}_{i}"
            try : 
                save_img(url, name)
            except:
                print(url)
                return
            
    elif loc['site'] == 'encar':
        for i in range(1,5):
            if loc['id'][3] == '0':
                tmp = '10'
            else:
                tmp = '0' + loc['id'][3]
            url = f"https://img.kcar.com/carpicture/carpicture{tmp}/pic{loc['id'][:4]}/kcar_{loc['id']}_00{i}.jpg"
            name = f"{loc['회사']}/{loc['모델']}/{loc['id']}_{i}"
            try : 
                save_img(url, name)
            except:
                print(url)
                return

In [17]:
for i in tqdm_notebook(range(len(id_df))):
    save_used_car_img(id_df.loc[i])

<ipython-input-17-d1730193efc4>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(id_df[1:10]))):


  0%|          | 0/9 [00:00<?, ?it/s]

<ipython-input-16-9dd4f3e99371>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loc['id'] = str(loc['id'])


https://ci.encar.com/carpicture/carpicture05/pic3055/30555229_001.jpg
https://ci.encar.com/carpicture/carpicture01/pic3031/30316479_001.jpg
https://ci.encar.com/carpicture/carpicture03/pic3013/30133562_001.jpg
https://ci.encar.com/carpicture/carpicture02/pic2902/29020109_001.jpg
